In [ ]:
pip install boto3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.9/13.9 MB 103.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.3/85.3 kB 6.8 MB/s eta 0:00:00


In [ ]:
import boto3

filename = "cardio_clean.csv"
df_predict.to_csv(filename, index=False)

aws_access_key = 'xxx'
aws_secret_key = 'xxx'
region = 'ap-southeast-1'
bucket_name = 'public-bucket-nhuttan-01'


s3 = boto3.client('s3',
                  aws_access_key_id=aws_access_key,
                  aws_secret_access_key=aws_secret_key,
                  region_name=region)

s3.upload_file(Filename=filename,
               Bucket=bucket_name,
               Key=f"data/processed/{filename}")

print(f" Đã upload {filename} lên bucket `{bucket_name}`")

#Khởi tạo SparkSession

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Cardio Prediction") \
    .getOrCreate()

# Đọc dữ liệu từ file CSV

In [ ]:
df_spark = spark.read.csv("cardio_clean.csv", header=True, inferSchema=True)
df_spark.printSchema()
df_spark.show(5)


root
 |-- age_years: integer (nullable = true)
 |-- height: integer (nullable = true)
 |-- weight: double (nullable = true)
 |-- ap_hi: double (nullable = true)
 |-- ap_lo: integer (nullable = true)
 |-- cholesterol: integer (nullable = true)
 |-- cardio: integer (nullable = true)
 |-- bmi: double (nullable = true)

+---------+------+------+-----+-----+-----------+------+------------------+
|age_years|height|weight|ap_hi|ap_lo|cholesterol|cardio|               bmi|
+---------+------+------+-----+-----+-----------+------+------------------+
|       50|   168|  62.0|110.0|   80|          1|     0|  21.9671201814059|
|       55|   156|  85.0|140.0|   90|          3|     1|34.927679158448385|
|       51|   165|  64.0|130.0|   70|          3|     1|23.507805325987146|
|       48|   169|  82.0|150.0|  100|          1|     1| 28.71047932495361|
|       47|   156|  56.0|100.0|   60|          1|     0|23.011176857330703|
+---------+------+------+-----+-----+-----------+------+------------------

##Tạo danh sách các cột feature đầu vào





In [ ]:
feature_cols = ['age_years', 'ap_hi', 'ap_lo', 'cholesterol', 'bmi']

#VectorAssembler: gộp các cột feature lại

In [ ]:
from pyspark.ml.feature import VectorAssembler, StandardScaler
assembler = VectorAssembler(inputCols=feature_cols, outputCol="assembled_features")

#StandardScaler: chuẩn hóa dữ liệu

In [ ]:
scaler = StandardScaler(inputCol="assembled_features", outputCol="features", withMean=True, withStd=True)

#Tách train/test

In [ ]:
train_data, test_data = df_spark.randomSplit([0.8, 0.2], seed=42)

# Hàm create_pipeline(model, use_scaler=True)

In [ ]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression

def create_pipeline(model, use_scaler=True):
    stages = [assembler]
    if use_scaler:
        stages.append(scaler)
        model.setFeaturesCol("features")
    else:
        model.setFeaturesCol("assembled_features")

    model.setLabelCol("cardio")
    stages.append(model)

    return Pipeline(stages=stages)

#Hàm đánh giá một mô hình đã predict

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

def evaluate_model(predictions, model_name="Mô hình"):
    evaluator_acc = MulticlassClassificationEvaluator(
        labelCol="cardio", predictionCol="prediction", metricName="accuracy"
    )
    evaluator_f1 = MulticlassClassificationEvaluator(
        labelCol="cardio", predictionCol="prediction", metricName="f1"
    )

    acc = evaluator_acc.evaluate(predictions)
    f1 = evaluator_f1.evaluate(predictions)

    print(f" {model_name}")
    print(f" Accuracy: {acc:.2%}")
    print(f" F1-score: {f1:.2%}\n")


#LogisticRegression

In [ ]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler, StandardScaler

assembler = VectorAssembler(inputCols=feature_cols, outputCol="assembled_features")
scaler = StandardScaler(inputCol="assembled_features", outputCol="features", withMean=True, withStd=True)
lr = LogisticRegression(featuresCol="features", labelCol="cardio", maxIter=100)

pipeline_lr = Pipeline(stages=[assembler, scaler, lr])

lr_model = pipeline_lr.fit(train_data)

lr_predictions = lr_model.transform(test_data)
evaluate_model(lr_predictions, "🔍 Logistic Regression")

 🔍 Logistic Regression
 Accuracy: 73.05%
 F1-score: 72.94%



#Decision Tree Classifier

In [ ]:
from pyspark.ml.classification import DecisionTreeClassifier
dt = DecisionTreeClassifier()
pipeline_dt = create_pipeline(dt, use_scaler=False)
dt_model = pipeline_dt.fit(train_data)
dt_predictions = dt_model.transform(test_data)
evaluate_model(dt_predictions, "Decision Tree")

 Decision Tree
 Accuracy: 73.38%
 F1-score: 73.30%



#Gradient Boosted Trees

In [ ]:
from pyspark.ml.classification import GBTClassifier
gbt = GBTClassifier(maxIter=50)
pipeline_gbt = create_pipeline(gbt, use_scaler=False)
gbt_model = pipeline_gbt.fit(train_data)
gbt_predictions = gbt_model.transform(test_data)
evaluate_model(gbt_predictions, "Gradient Boosted Trees")

 Gradient Boosted Trees
 Accuracy: 73.63%
 F1-score: 73.57%



#Linear SVM (Support Vector Machine)

In [ ]:
from pyspark.ml.classification import LinearSVC
svm = LinearSVC(maxIter=100)
pipeline_svm = create_pipeline(svm, use_scaler=True)
svm_model = pipeline_svm.fit(train_data)
svm_predictions = svm_model.transform(test_data)
evaluate_model(svm_predictions, "Linear SVM")

 Linear SVM
 Accuracy: 72.94%
 F1-score: 72.70%



#Multilayer Perceptron (MLP)

In [ ]:
from pyspark.ml.classification import MultilayerPerceptronClassifier

layers = [5, 5, 2]

mlp = MultilayerPerceptronClassifier(maxIter=100, layers=layers, seed=42, featuresCol="features", labelCol="cardio")

pipeline_mlp = create_pipeline(mlp, use_scaler=True)
mlp_model = pipeline_mlp.fit(train_data)
mlp_predictions = mlp_model.transform(test_data)

evaluate_model(mlp_predictions, "Multilayer Perceptron (MLP)")


 Multilayer Perceptron (MLP)
 Accuracy: 73.68%
 F1-score: 73.63%



#Mô hình One-vs-Rest với Linear SVM (OvR-SVM)

In [ ]:
from pyspark.ml.classification import OneVsRest

# Sử dụng LinearSVC làm base classifier
ovr = OneVsRest(classifier=LinearSVC(maxIter=100))
pipeline_ovr = create_pipeline(ovr, use_scaler=True)

ovr_model = pipeline_ovr.fit(train_data)
ovr_predictions = ovr_model.transform(test_data)

evaluate_model(ovr_predictions, "One-vs-Rest (Linear SVM)")


 One-vs-Rest (Linear SVM)
 Accuracy: 72.94%
 F1-score: 72.70%

